<h1><a href="#part3">click to go part # 3</a></h1>
<h1>List of Postal Code in the Toronto city, Canada</h1>
<i>The first step consist in import required Python libraries</i>

In [8]:
# import required libs
import requests
from bs4 import BeautifulSoup
import pandas as pd

<i>Next requesting the wikipedia uri</i>

In [9]:
# requesting uri
wikilink = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(wikilink)

<i>Then, with <b>bs4</b> lib, we read the content, extract the first html-table and read it with <b>pandas</b> </i>

In [10]:
soup  = BeautifulSoup(res.content, "lxml") # read/parse html-content
wikitable = soup.find_all("table")[0] # get first html-table
wk_df = pd.read_html(str(wikitable))[0] # read/convert to pandas dataframe

<i>Now, let's start cleaning and adjusting our dataframe</i>

In [11]:
wk_df.columns= ['PostalCode', 'Borough', 'Neighbourhood'] # columns header for our dataframe
wk_df.drop(0, inplace=True) # the first row have we header column name, the drop it
wk_df = wk_df[wk_df.Borough != 'Not assigned'] # if Borough have Not assigned string then ignore

# locate Neighbourhood with 'Not assigned' string then replace with the Borough string content
wk_df.loc[wk_df.Neighbourhood == 'Not assigned', 'Neighbourhood'] = wk_df.Borough

# at this point we need group PostalCode and Borough with the same name, then apply the magic!
# join Neighbourhood belong to repeated PostalCode, combined into one row 
wk_df = wk_df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

<i>Print top 15 rows</i>

In [12]:
# print 15 first rows
wk_df.head(-15)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
83,M6R,West Toronto,"Parkdale, Roncesvalles"
84,M6S,West Toronto,"Runnymede, Swansea"
85,M7A,Queen's Park,Queen's Park
86,M7R,Mississauga,Canada Post Gateway Processing Centre


In [13]:
# shape of dataframe
wk_df.shape

(103, 3)

<h1><a id="part2">Init of part # 2</a></h1>
<i>First step consist in download the geo coordinates file</i>

In [3]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print ('File is here!')

File is here!


<i>Then read the csv files with pandas awesome lib</i>

In [14]:
geo_df = pd.read_csv('Geospatial_Coordinates.csv')
geo_df.head(-15)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
83,M6R,43.648960,-79.456325
84,M6S,43.651571,-79.484450
85,M7A,43.662301,-79.389494
86,M7R,43.636966,-79.615819


<i>Next step consist in asign latitude and longitude to our dataframe</i>

In [15]:
wk_df['Latitude'] = geo_df.Latitude
wk_df['Longitude'] = geo_df.Longitude

In [16]:
wk_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<h1><a id="part3">Init of part # 3</a></h1>

In [40]:
# import map and geo locates libs
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

<i>Determine latotude and longitude of Toronto city</i> <br/>
<i>We are using geolocator lib for this task</i>

In [38]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


<i>Last step is about iterate pandas dataframe for extract some variables for draw the map with Market of each Borough over the map</i>

In [39]:
# initialize the map
map_ontario = folium.Map(location=[latitude, longitude], zoom_start=10)

# iterate dataframe
for row in wk_df.iterrows():
    # get all variables:
    lat = row[1]['Latitude']
    lng = row[1]['Longitude']
    borough = row[1]['Borough']
    neighborhood = row[1]['Neighbourhood']
    postalcode = row[1]['PostalCode']
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    
    # add markers
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ontario) 

map_ontario 